In [2]:
%load_ext autoreload
%autoreload 2

import os
import char_max_likelihood
import metrics
import pandas as pd
import LSTMGenerator

Importing Jupyter notebook from lstm_words.ipynb


# Generate Speeches

In [3]:
presidents = ["obama", "bush", "reagan", "trump"]
n=2

## LSTM

In [3]:
for pres in presidents:
    lstm = LSTMGenerator.LSTMGenerator(pres, force_cpu=True)
    speeches = lstm.generate_n(n, length=5000)
    lstm.persist(speeches)

/home/gabriel/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## CharPred

In [48]:
order = 10
letters = 5000*5
filepath = "../data/charpred/{}_generated/{}.txt"

for pres in presidents:
    lm = char_max_likelihood.train_char_lm(pres, order)
    for i in range(0,n):
        s = char_max_likelihood.generate_text(lm, order, letters)
        if not os.path.exists("../data/charpred/{}_generated/".format(pres)):
            os.makedirs("../data/charpred/{}_generated/".format(pres))
        with open(filepath.format(pres,i), "w") as f:
            f.write(s)

## NGram

In [ ]:
# todo generate speeches and save to ../data/ngram/{}_generated/

# Qualitative Evaluation

In [47]:
# print some speeches here
for loc in generated_speech_locations:
    speech = open("../data/"+loc+"/"+presidents[1]+"_generated/0.txt").read()
    print("Generated by " + loc+":")
    print(speech[:700])
    print("\n-------------------------------------\n")

Generated by lstm:
good evening ladies and gentlemen a lot of a better time . the president yes to the iraqi government will be able to the united states . and the united states of the world is to do the world . the world is to be a new challenge . and this will be a new nation . we must continue to work . and the last year that we have been a way to live in the world . and i ask congress to do you to be a lot of people . and so i 'm not a new man . we will not be accomplished with the united states . i ask you to work . i have brought a lot of freedom . the president i have seen the world . i ask congress to join me in the american people . we will make sure that the united states . and we will work alongside

-------------------------------------

Generated by charpred:
Madam Speaker, Vice President having to worry about?

THE PRESIDENT: Eight years. You used to be known as the sun sets on this issue. The funding. This year, we will not leave their employees to set up health savings 

# Quantitative Evaluation

In [7]:
metric_list = ["tfidf_cosine", "tfidf_distance", "rouge", "mean_sentence_len (gen)", "mean_sentence_len (orig)", "mean_word_len (gen)", "mean_word_len (orig)"]
generated_speech_locations = ["lstm", "charpred"]#, "ngram"]

In [8]:
def generate_metrics(president):
    results = pd.DataFrame(columns=metric_list)
    for loc in generated_speech_locations:
        mean_cosine, _, _ = metrics.get_cosine_sim_tfidf(president, loc+"/"+president+"_generated", None, print_results=False)
        mean_distance = metrics.get_top_n_rank_distance(president, loc+"/"+president+"_generated", None, 15)
        mean_rouge, _, _ = metrics.get_rouge_score(president, loc+"/"+president+"_generated", None, print_results=False)
        mean_sentence_l_g = metrics.calculate_mean_sentence_length(loc+"/"+president+"_generated")
        mean_sentence_l = metrics.calculate_mean_sentence_length(president)
        mean_word_l_g = metrics.calculate_mean_word_length(loc+"/"+president+"_generated")
        mean_word_l = metrics.calculate_mean_word_length(president)
        results=results.append({"tfidf_cosine":mean_cosine,
                        "tfidf_distance":mean_distance,
                        "rouge":mean_rouge,
                        "mean_sentence_len (gen)":mean_sentence_l_g,
                        "mean_sentence_len (orig)":mean_sentence_l,
                        "mean_word_len (gen)":mean_word_l_g,
                        "mean_word_len (orig)":mean_word_l
                               }, ignore_index=True)

    results.index = generated_speech_locations
    return results

## Obama

In [9]:
results = generate_metrics("obama")
results

,tfidf_cosine,tfidf_distance,rouge,mean_sentence_len (gen),mean_sentence_len (orig),mean_word_len (gen),mean_word_len (orig)
lstm,0.046214,601.266998,0.407979,11.772116,18.303574,3.312004,4.624454
charpred,0.091898,3003.545199,0.572537,17.162455,18.303574,4.631718,4.624454


## Bush

In [10]:
results = generate_metrics("bush")
results

,tfidf_cosine,tfidf_distance,rouge,mean_sentence_len (gen),mean_sentence_len (orig),mean_word_len (gen),mean_word_len (orig)
lstm,0.056604,363.995729,0.353213,10.978056,17.351306,3.841696,4.79439
charpred,0.092202,3021.826087,0.487724,17.332039,17.351306,4.775594,4.79439


In [31]:
results.to_markdown()

'|          |   tfidf_cosine |   tfidf_distance |    rouge |     msl |     mwl |\n|:---------|---------------:|-----------------:|---------:|--------:|--------:|\n| lstm     |      0.0566038 |          363.996 | 0.353213 | 10.9781 | 3.8417  |\n| charpred |      0.092398  |         3036.47  | 0.48746  | 17.1922 | 4.84861 |'

## Reagan

In [11]:
results = generate_metrics("reagan")
results

,tfidf_cosine,tfidf_distance,rouge,mean_sentence_len (gen),mean_sentence_len (orig),mean_word_len (gen),mean_word_len (orig)
lstm,0.040750,326.732364,0.346520,9.771818,18.456017,3.716980,4.720304
charpred,0.088859,3334.680667,0.520757,17.612524,18.456017,4.710816,4.720304


## Trump

In [12]:
results = generate_metrics("trump")
results

,tfidf_cosine,tfidf_distance,rouge,mean_sentence_len (gen),mean_sentence_len (orig),mean_word_len (gen),mean_word_len (orig)
lstm,0.035442,208.641637,0.284851,10.834158,12.49461,2.828583,4.574695
charpred,0.082566,3048.083968,0.478450,12.185473,12.49461,4.572421,4.574695
